In [1]:
import requests
from pandas.io.json import json_normalize
import pandas as pd
import math
import time

In [2]:
#https://api.mercadolibre.com/classified_locations/countries/MX --> devuelve los estados de un country
#https://api.mercadolibre.com/classified_locations/states/TUxNUERJUzYwOTQ --> devuelve las ciudades de un estado
#https://api.mercadolibre.com/classified_locations/cities/TUxNQ0JFTjM2MjQ --> devuelve neighborhoods de una ciudad 
#https://api.mercadolibre.com/classified_locations/neighborhoods/TUxNQk7BUDUwNzQ <-- devuelve sub neighborhoods si existieran

#https://api.mercadolibre.com/sites/MLM/categories ---> todas las categorias disponibles


state = "TUxNUERJUzYwOTQ"
city = "TUxNQ0JFTjM2MjQ"
neighborhoods = "TUxNQ0JFTjM2MjQ"
categoria = "MLM1144"

#r = requests.get("https://api.mercadolibre.com/sites/MLM/search?category=" + categoria + "&state=" + state + "&city=" + city)
#print(r.text)

In [3]:
contador = 0
offset = 50
results = []
#secondary_results = []

while(True):
    pagina = offset* contador
    limit = pagina + 49
    if(pagina>1000):
        break                
    r = requests.get("https://api.mercadolibre.com/sites/MLM/search?category=" + categoria + "&state=" + state + "&city=" + city + "&neighborhoods=" + neighborhoods + "&offset="+str(pagina))
    data = r.json()
    #print(data)
    paging = data.get('paging')
    total_resultados = paging['total']
    results += data.get('results')
    #secondary_results += data.get('secondary_results')
    paginas = math.floor(total_resultados/offset)
    if(contador == paginas):
        break
    contador += 1

In [5]:
df = pd.DataFrame.from_dict(json_normalize(results), orient='columns').drop(['attributes','title'],1)

In [6]:
df.head()

,accepts_mercadopago,address.city_id,address.city_name,address.state_id,address.state_name,available_quantity,buying_mode,catalog_product_id,category_id,condition,...,shipping.free_shipping,shipping.logistic_type,shipping.mode,shipping.store_pick_up,shipping.tags,site_id,sold_quantity,stop_time,tags,thumbnail
0,True,TUxNQ0JFTjM2MjQ,Benito Juárez,MX-DIF,Distrito Federal,250,buy_it_now,None,MLM187264,new,...,True,drop_off,me2,False,[mandatory_free_shipping],MLM,500,2038-02-15T04:00:00.000Z,"[brand_verified, good_quality_picture, good_qu...",http://mlm-s2-p.mlstatic.com/693802-MLM2815602...
1,True,TUxNQ0JFTjM2MjQ,Benito Juárez,MX-DIF,Distrito Federal,100,buy_it_now,None,MLM191011,new,...,True,drop_off,me2,False,[mandatory_free_shipping],MLM,250,2039-03-07T22:56:02.000Z,"[good_quality_picture, good_quality_thumbnail,...",http://mlm-s2-p.mlstatic.com/700289-MLM2856076...
2,True,TUxNQ0JFTjM2MjQ,Benito Juárez,MX-DIF,Distrito Federal,1,buy_it_now,MLM6159489,MLM90891,new,...,False,drop_off,me2,False,[],MLM,100,2036-11-27T14:24:55.000Z,"[good_quality_picture, good_quality_thumbnail,...",http://mlm-s2-p.mlstatic.com/731830-MLM2935512...
3,True,TUxNQ1RMQTY3NDM,Tlalpan,MX-DIF,Distrito Federal,50,buy_it_now,MLM9894312,MLM8498,new,...,True,drop_off,me2,False,[],MLM,50,2039-01-23T04:00:00.000Z,"[brand_verified, good_quality_picture, good_qu...",http://mlm-s2-p.mlstatic.com/669463-MLM2926324...
4,True,TUxNQ0JFTjM2MjQ,Benito juárez,MX-DIF,Distrito Federal,200,buy_it_now,None,MLM156065,new,...,False,not_specified,not_specified,False,[],MLM,50,2039-01-29T04:00:00.000Z,"[good_quality_picture, good_quality_thumbnail,...",http://mlm-s1-p.mlstatic.com/786840-MLM2984268...


In [7]:
df.to_csv('productos.csv')